In [1]:
import torch
import numpy as np
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pickle

class SkeletonDataset(Dataset):
    """
    Custom Dataset cho dữ liệu skeleton.
    Đọc dữ liệu từ file .npy và label từ file .pkl,
    sau đó định hình lại cho phù hợp với mô hình RNN/GRU.
    """
    def __init__(self, data_path, label_path):
        # Tải label
        with open(label_path, 'rb') as f:
            self.labels = pickle.load(f)

        # Tải dữ liệu và chuyển sang tensor
        # Dữ liệu gốc có shape (N, C, T, V, M)
        raw_data = np.load(data_path)
        self.data = torch.from_numpy(raw_data).float()

        # Thông tin shape
        self.N, self.C, self.T, self.V, self.M = self.data.shape
        print(f"Loaded data from {data_path} with shape: ({self.N}, {self.C}, {self.T}, {self.V}, {self.M})")


    def __len__(self):
        return self.N

    def __getitem__(self, index):
        # Lấy dữ liệu và label tại index
        sample_data = self.data[index] # Shape: (C, T, V, M)
        label = self.labels[index]

        # Thay đổi shape để phù hợp với GRU
        # Chuyển (C, T, V, M) -> (T, C, V, M)
        sample_data = sample_data.permute(1, 0, 2, 3)

        # Làm phẳng các chiều C, V, M thành một vector đặc trưng duy nhất cho mỗi bước thời gian T
        # Shape sau khi làm phẳng: (T, C * V * M)
        # Ví dụ: (20, 3 * 61 * 1) = (20, 183)
        flattened_data = sample_data.reshape(self.T, -1)

        return flattened_data, torch.tensor(label, dtype=torch.long)

In [3]:
import random
def set_seed(seed_value):
    random.seed(seed_value) # Python's random module
    np.random.seed(seed_value) # NumPy
    torch.manual_seed(seed_value) # PyTorch on CPU
    torch.cuda.manual_seed_all(seed_value) # PyTorch on all GPUs (if available)
    
    # Cấu hình cho thuật toán cuDNN (thường được sử dụng bởi PyTorch trên GPU)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    print(f"Global seed set to {seed_value}")

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class AdvancedGRUModel(nn.Module):
    def __init__(self, input_size, num_classes):
        super(AdvancedGRUModel, self).__init__()

        # --- Lớp GRU ---
        # Keras: GRU(128, return_sequences=True, activation='relu')
        # PyTorch: nn.GRU không hỗ trợ trực tiếp activation='relu'. 
        # Chúng ta sẽ sử dụng nn.GRU tiêu chuẩn và thảo luận về điểm này bên dưới.
        self.gru1 = nn.GRU(input_size=input_size, hidden_size=128, batch_first=True)
        self.dropout1 = nn.Dropout(0)

        self.gru2 = nn.GRU(input_size=128, hidden_size=128, batch_first=True)
        self.dropout2 = nn.Dropout(0)

        self.gru3 = nn.GRU(input_size=128, hidden_size=256, batch_first=True)
        self.dropout3 = nn.Dropout(0)
        
        self.gru4 = nn.GRU(input_size=256, hidden_size=512, batch_first=True)
        self.dropout4 = nn.Dropout(0)

        # Lớp GRU cuối cùng, tương đương return_sequences=False
        self.gru5 = nn.GRU(input_size=512, hidden_size=512, batch_first=True)

        # --- Khối Classifier (MLP) ---
        # Sử dụng nn.Sequential để nhóm các lớp Dense lại cho gọn gàng
        self.classifier = nn.Sequential(
            nn.BatchNorm1d(512),       # Tương đương BatchNormalization() sau GRU
            nn.Linear(512, 512),
            nn.ReLU(),                 # Tương đương activation='relu'
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, num_classes) # Lớp output, không có softmax (xem giải thích)
        )

    def forward(self, x):
        # Luồng dữ liệu qua các lớp GRU
        # self.gru(x) trả về (output, h_n)
        # output có shape (batch, seq_len, hidden_size)
        # h_n là hidden state cuối cùng
        
        # return_sequences=True
        out, _ = self.gru1(x)
        out = self.dropout1(out)
        
        out, _ = self.gru2(out)
        out = self.dropout2(out)

        out, _ = self.gru3(out)
        out = self.dropout3(out)

        out, _ = self.gru4(out)
        out = self.dropout4(out)

        # Lớp GRU cuối, tương đương return_sequences=False
        out, _ = self.gru5(out)
        
        # Lấy output của bước thời gian cuối cùng
        out = out[:, -1, :] # Shape: (batch, hidden_size)

        # Cho qua khối classifier
        out = self.classifier(out)
        
        return out

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
checkpoint_path = '/workspace/data/log/hdf5/advanced_gru_model.pth'

# 2. Khởi tạo model với đúng input_size và num_classes
input_size  = 3 * 61 * 1    # giữ nguyên như lúc train
num_classes = 2139          # số lớp của bạn
model = AdvancedGRUModel(input_size=input_size, num_classes=num_classes)
model.to(device)

# 1. Load checkpoint (có thể là dict hoặc có key 'state_dict')
ckpt = torch.load(checkpoint_path, map_location=device)
pretrained_dict = ckpt.get('state_dict', ckpt)

# 2. Lấy state_dict hiện tại của model
model_dict = model.state_dict()

# 3. Lọc ra những keys vừa tồn tại, vừa cùng shape
filtered_dict = {
    k: v for k, v in pretrained_dict.items()
    if k in model_dict and v.shape == model_dict[k].shape
}

# 4. Update weights và load
model_dict.update(filtered_dict)
model.load_state_dict(model_dict)

# 5. Chuyển sang inference mode
model.eval()


AdvancedGRUModel(
  (gru1): GRU(183, 128, batch_first=True)
  (dropout1): Dropout(p=0, inplace=False)
  (gru2): GRU(128, 128, batch_first=True)
  (dropout2): Dropout(p=0, inplace=False)
  (gru3): GRU(128, 256, batch_first=True)
  (dropout3): Dropout(p=0, inplace=False)
  (gru4): GRU(256, 512, batch_first=True)
  (dropout4): Dropout(p=0, inplace=False)
  (gru5): GRU(512, 512, batch_first=True)
  (classifier): Sequential(
    (0): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Linear(in_features=512, out_features=512, bias=True)
    (2): ReLU()
    (3): Linear(in_features=512, out_features=256, bias=True)
    (4): ReLU()
    (5): Linear(in_features=256, out_features=256, bias=True)
    (6): ReLU()
    (7): Linear(in_features=256, out_features=128, bias=True)
    (8): ReLU()
    (9): Linear(in_features=128, out_features=64, bias=True)
    (10): ReLU()
    (11): Linear(in_features=64, out_features=2139, bias=True)
  )
)

In [6]:
DATA_DIR = '/workspace/data/npy_100/transformed/mediapipe61'

# Chỉ cần loader cho tập bạn muốn đánh giá
test_dataset = SkeletonDataset(
    f'{DATA_DIR}/test_data.npy',
    f'{DATA_DIR}/test_label.pkl'
)
test_loader = DataLoader(
    test_dataset,
    batch_size=256,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

Loaded data from /workspace/data/npy_100/transformed/mediapipe61/test_data.npy with shape: (85560, 3, 24, 61, 1)


In [7]:
import torch.nn as nn
from tqdm import tqdm

criterion = nn.CrossEntropyLoss()

def inference_and_report(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for inputs, labels in tqdm(dataloader, desc="Inference"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * labels.size(0)

            preds = outputs.argmax(dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())

    avg_loss = running_loss / len(dataloader.dataset)
    accuracy = (np.array(all_preds) == np.array(all_labels)).mean()

    print(f"\nInference Results → Loss: {avg_loss:.4f}, Acc: {accuracy:.4f}\n")
    print("=== Classification Report ===")
    print(classification_report(all_labels, all_preds, digits=4))
    print("\n=== Confusion Matrix (shape {}) ===".format(
        confusion_matrix(all_labels, all_preds).shape
    ))
    # Nếu bạn không cần in ma trận đầy đủ, có thể bỏ phần này.

# Cuối cùng, chỉ cần gọi:
inference_and_report(model, test_loader, criterion, device)


Inference: 100%|██████████| 335/335 [00:03<00:00, 109.07it/s]
/root/miniconda3/envs/cre/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/root/miniconda3/envs/cre/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/root/miniconda3/envs/cre/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn


Inference Results → Loss: 11.3666, Acc: 0.0004

=== Classification Report ===
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000        40
           1     0.0000    0.0000    0.0000        40
           2     0.0000    0.0000    0.0000        40
           3     0.0000    0.0000    0.0000        40
           4     0.0000    0.0000    0.0000        40
           5     0.0000    0.0000    0.0000        40
           6     0.0000    0.0000    0.0000        40
           7     0.0000    0.0000    0.0000        40
           8     0.0000    0.0000    0.0000        40
           9     0.0000    0.0000    0.0000        40
          10     0.0000    0.0000    0.0000        40
          11     0.0000    0.0000    0.0000        40
          12     0.0000    0.0000    0.0000        40
          13     0.0000    0.0000    0.0000        40
          14     0.0000    0.0000    0.0000        40
          15     0.0000    0.0000    0.0000        40
  

In [12]:
def inference_and_report(model, dataloader, criterion, device, num_samples_to_print=20):

    model.eval()
    running_loss = 0.0
    all_preds, all_labels = [], []

    # Tắt việc tính toán gradient để tăng tốc độ
    with torch.no_grad():
        for inputs, labels in tqdm(dataloader, desc="Inference"):
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Lấy đầu ra của mô hình
            outputs = model(inputs)
            
            # Tính loss
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)

            # Lấy dự đoán (chỉ số của giá trị lớn nhất)
            _, preds = torch.max(outputs, 1)
            
            # Chuyển tensor sang numpy và thêm vào danh sách
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Tính toán loss trung bình và độ chính xác tổng thể
    avg_loss = running_loss / len(dataloader.dataset)
    accuracy = (np.array(all_preds) == np.array(all_labels)).mean()

    # In kết quả tổng hợp
    print(f"\nInference Results → Loss: {avg_loss:.4f}, Acc: {accuracy:.4f}\n")
    
    # In báo cáo phân loại chi tiết
    # print("=== Classification Report ===")
    # Thêm `zero_division=0` để tránh lỗi khi một lớp không có trong dự đoán
    # print(classification_report(all_labels, all_preds, digits=4, zero_division=0)) 
    
    # In ma trận nhầm lẫn
    # print("\n=== Confusion Matrix (shape: {}) ===".format(
    #     confusion_matrix(all_labels, all_preds).shape
    # ))
    # # print(confusion_matrix(all_labels, all_preds)) # Bỏ comment nếu muốn in ma trận đầy đủ

    # --- PHẦN MỚI: In ra dự đoán và ground truth ---
    print("\n=== Sample Predictions vs. Ground Truth ===")
    # Giới hạn số lượng in ra để tránh làm đầy console
    num_to_show = min(len(all_preds), num_samples_to_print)
    print(f"Showing first {num_to_show} samples:\n")
    print(f"{'Sample':<10} | {'Prediction':<12} | {'Ground Truth':<12}")
    print("-" * 40)
    for i in range(num_to_show):
        print(f"{i+1:<10} | {all_preds[i]:<12} | {all_labels[i]:<12}")


# # Cuối cùng, chỉ cần gọi:
inference_and_report(model, test_loader, criterion, device, num_samples_to_print=41)

Inference: 100%|██████████| 335/335 [00:02<00:00, 116.02it/s]


Inference Results → Loss: 11.3666, Acc: 0.0004


=== Sample Predictions vs. Ground Truth ===
Showing first 41 samples:

Sample     | Prediction   | Ground Truth
----------------------------------------
1          | 388          | 0           
2          | 1670         | 0           
3          | 1670         | 0           
4          | 1670         | 0           
5          | 1670         | 0           
6          | 1670         | 0           
7          | 1670         | 0           
8          | 1670         | 0           
9          | 1670         | 0           
10         | 1670         | 0           
11         | 1670         | 0           
12         | 1670         | 0           
13         | 1670         | 0           
14         | 1670         | 0           
15         | 1670         | 0           
16         | 1670         | 0           
17         | 1670         | 0           
18         | 1670         | 0           
19         | 1670         | 0           
20         | 1670 

In [13]:
def inference_and_report(model, dataloader, criterion, device, class_names, num_samples_to_print=20):
    model.eval()
    running_loss = 0.0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for inputs, labels in tqdm(dataloader, desc="Inference"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_loss = running_loss / len(dataloader.dataset)
    accuracy = (np.array(all_preds) == np.array(all_labels)).mean()

    print(f"\nInference Results → Loss: {avg_loss:.4f}, Acc: {accuracy:.4f}\n")
    
    # === PHẦN MỚI: Sử dụng class_names trong classification_report ===
    print("=== Classification Report ===")
    # Thêm `target_names` để báo cáo hiển thị tên lớp, dễ đọc hơn
    # print(classification_report(
    #     all_labels, 
    #     all_preds, 
    #     target_names=class_names, 
    #     digits=4, 
    #     zero_division=0
    # ))
    
    # print("\n=== Confusion Matrix (shape: {}) ===".format(
    #     confusion_matrix(all_labels, all_preds).shape
    # ))

    # === PHẦN MỚI: In ra dự đoán và ground truth với tên lớp ===
    print("\n=== Sample Predictions vs. Ground Truth ===")
    num_to_show = min(len(all_preds), num_samples_to_print)
    print(f"Showing first {num_to_show} samples:\n")
    # Điều chỉnh độ rộng cột cho phù hợp với độ dài của tên lớp
    print(f"{'Sample':<10} | {'Prediction':<25} | {'Ground Truth':<25}")
    print("-" * 65)
    for i in range(num_to_show):
        pred_class_name = class_names[all_preds[i]]
        label_class_name = class_names[all_labels[i]]
        print(f"{i+1:<10} | {pred_class_name:<25} | {label_class_name:<25}")

# --- CÁCH SỬ DỤNG ---

# 1. Tải tệp classname.npy
try:
    # allow_pickle=True cần thiết khi tải mảng chứa các đối tượng, ví dụ như chuỗi ký tự
    class_names = np.load('/workspace/npy_converted_full/transformed/mediapipe61/class_names.npy', allow_pickle=True)
except FileNotFoundError:
    print("Lỗi: Không tìm thấy tệp 'classname.npy'. Hãy đảm bảo tệp này nằm trong cùng thư mục.")
    exit() # Thoát nếu không có file classname

inference_and_report(model, test_loader, criterion, device, class_names=class_names, num_samples_to_print=50)

Inference: 100%|██████████| 335/335 [00:02<00:00, 116.08it/s]


Inference Results → Loss: 11.3666, Acc: 0.0004

=== Classification Report ===

=== Sample Predictions vs. Ground Truth ===
Showing first 50 samples:

Sample     | Prediction                | Ground Truth             
-----------------------------------------------------------------
1          | chân_dung                 | 7up                      
2          | mỳ_tôm_T                  | 7up                      
3          | mỳ_tôm_T                  | 7up                      
4          | mỳ_tôm_T                  | 7up                      
5          | mỳ_tôm_T                  | 7up                      
6          | mỳ_tôm_T                  | 7up                      
7          | mỳ_tôm_T                  | 7up                      
8          | mỳ_tôm_T                  | 7up                      
9          | mỳ_tôm_T                  | 7up                      
10         | mỳ_tôm_T                  | 7up                      
11         | mỳ_tôm_T                  | 7up  

In [14]:
def inference_and_report(model, dataloader, criterion, device, 
                         pred_class_names, gt_class_names, 
                         num_samples_to_print=20):
    model.eval()
    running_loss = 0.0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for inputs, labels in tqdm(dataloader, desc="Inference"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_loss = running_loss / len(dataloader.dataset)
    
    accuracy = (np.array(all_preds) == np.array(all_labels)).mean()
    print(f"\nInference Results → Loss: {avg_loss:.4f}, Accuracy (by index): {accuracy:.4f}\n")

    # --- In ra dự đoán và ground truth với 2 bộ tên lớp riêng biệt ---
    print("\n=== Sample Predictions vs. Ground Truth ===")
    num_to_show = min(len(all_preds), num_samples_to_print)
    print(f"Showing first {num_to_show} samples:\n")
    print(f"{'Sample':<10} | {'Prediction Class':<30} | {'Ground Truth Class':<30}")
    print("-" * 75)
    for i in range(num_to_show):
        # Sử dụng đúng tệp class_names cho từng loại nhãn
        pred_class_name = pred_class_names[all_preds[i]]
        label_class_name = gt_class_names[all_labels[i]]
        print(f"{i+1:<10} | {pred_class_name:<30} | {label_class_name:<30}")


PRED_CLASS_NAMES_PATH = '/workspace/npy_converted_full/transformed/mediapipe61/class_names.npy'
GT_CLASS_NAMES_PATH = '/workspace/data/npy_100/transformed/mediapipe61/class_names.npy' 

try:
    pred_class_names = np.load(PRED_CLASS_NAMES_PATH, allow_pickle=True)
    gt_class_names = np.load(GT_CLASS_NAMES_PATH, allow_pickle=True)
except FileNotFoundError as e:
    print(f"Lỗi: Không tìm thấy tệp .npy. Vui lòng kiểm tra lại đường dẫn.")
    print(f"Chi tiết lỗi: {e}")
    exit()

# 2. Gọi hàm inference và truyền cả 2 danh sách tên lớp vào
# Giả sử model, test_loader, criterion, device đã được định nghĩa
inference_and_report(
    model, 
    test_loader, 
    criterion, 
    device, 
    pred_class_names=pred_class_names, 
    gt_class_names=gt_class_names, 
    num_samples_to_print=50
)

Inference: 100%|██████████| 335/335 [00:02<00:00, 114.94it/s]


Inference Results → Loss: 11.3666, Accuracy (by index): 0.0004


=== Sample Predictions vs. Ground Truth ===
Showing first 50 samples:

Sample     | Prediction Class               | Ground Truth Class            
---------------------------------------------------------------------------
1          | chân_dung                      | 7                             
2          | mỳ_tôm_T                       | 7                             
3          | mỳ_tôm_T                       | 7                             
4          | mỳ_tôm_T                       | 7                             
5          | mỳ_tôm_T                       | 7                             
6          | mỳ_tôm_T                       | 7                             
7          | mỳ_tôm_T                       | 7                             
8          | mỳ_tôm_T                       | 7                             
9          | mỳ_tôm_T                       | 7                             
10         | mỳ_t

In [ ]:
def inference_and_report(model, dataloader, criterion, device, 
                         pred_class_names, gt_class_names, 
                         print_range=(0, 15)): # <-- Tham số mới
    model.eval()
    running_loss = 0.0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for inputs, labels in tqdm(dataloader, desc="Inference"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_loss = running_loss / len(dataloader.dataset)
    accuracy = (np.array(all_preds) == np.array(all_labels)).mean()
    print(f"\nInference Results → Loss: {avg_loss:.4f}, Accuracy (by index): {accuracy:.4f}\n")

    # --- PHẦN ĐƯỢC CẬP NHẬT: In theo khoảng ---
    print("\n=== Sample Predictions vs. Ground Truth ===")
    
    start_idx, end_idx = print_range
    total_samples = len(all_preds)

    # Kiểm tra để đảm bảo khoảng in hợp lệ
    if not (0 <= start_idx < end_idx <= total_samples):
        print(f"Lỗi: Khoảng in [{start_idx}, {end_idx}) không hợp lệ. Tổng số mẫu là {total_samples}.")
        # In 15 mẫu đầu tiên như mặc định nếu có lỗi
        start_idx, end_idx = 0, min(15, total_samples)
        
    print(f"Showing samples from index {start_idx} to {end_idx - 1}:\n")
    print(f"{'Sample #':<10} | {'Prediction Class':<30} | {'Ground Truth Class':<30}")
    print("-" * 75)

    # Lặp qua đúng khoảng chỉ số đã cho
    for i in range(start_idx, end_idx):
        pred_class_name = pred_class_names[all_preds[i]]
        label_class_name = gt_class_names[all_labels[i]]
        print(f"{i + 1:<10} | {pred_class_name:<30} | {label_class_name:<30}")

PRED_CLASS_NAMES_PATH = '/workspace/npy_converted_full/transformed/mediapipe61/class_names.npy'
GT_CLASS_NAMES_PATH = '/workspace/data/npy_100/transformed/mediapipe61/class_names.npy' 

try:
    pred_class_names = np.load(PRED_CLASS_NAMES_PATH, allow_pickle=True)
    gt_class_names = np.load(GT_CLASS_NAMES_PATH, allow_pickle=True)
except FileNotFoundError as e:
    print(f"Lỗi: Không tìm thấy tệp .npy. Vui lòng kiểm tra lại đường dẫn.")
    print(f"Chi tiết lỗi: {e}")
    exit()

a = 15

inference_and_report(
    model, 
    test_loader, 
    criterion, 
    device, 
    pred_class_names=pred_class_names, 
    gt_class_names=gt_class_names, 
    print_range=(a, a + 15)
)

Inference: 100%|██████████| 335/335 [00:02<00:00, 114.93it/s]


Inference Results → Loss: 11.3666, Accuracy (by index): 0.0004


=== Sample Predictions vs. Ground Truth ===
Showing samples from index 15 to 29:

Sample #   | Prediction Class               | Ground Truth Class            
---------------------------------------------------------------------------
16         | mỳ_tôm_T                       | 7                             
17         | mỳ_tôm_T                       | 7                             
18         | mỳ_tôm_T                       | 7                             
19         | mỳ_tôm_T                       | 7                             
20         | mỳ_tôm_T                       | 7                             
21         | mỳ_tôm_T                       | 7                             
22         | chân_dung                      | 7                             
23         | chân_dung                      | 7                             
24         | chân_dung                      | 7                             
25    